<a href="https://colab.research.google.com/github/arpdm/apcs/blob/main/apcs_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pathlib
import tarfile
import urllib.request

In [2]:
core_path = ""
data_path = "data/smass_data/"

# Set the path for downloading data files
pathlib.Path(os.path.join(core_path, data_path)).mkdir(parents=True, exist_ok=True)

In [3]:
smass_data_files = \
    {'file1': {'url': 'http://smass.mit.edu/data/smass/Bus.Taxonomy.txt',
               'sha256': '0ce970a6972dd7c49d512848b9736d00b621c9d6395a035bd1b4f3780d4b56c6'},
     'file2': {'url': 'http://smass.mit.edu/data/smass/smass2data.tar.gz',
               'sha256': 'dacf575eb1403c08bdfbffcd5dbfe12503a588e09b04ed19cc4572584a57fa97'}}

In [4]:
def downloadDataFiles(file_dictionary):

    for df in file_dictionary:

        # Get the URL and create a download filepath by splitting it at the last "/"
        split = urllib.parse.urlsplit(file_dictionary[df]["url"])
        filename = pathlib.Path(os.path.join(core_path, data_path, split.path.split("/")[-1]))

        # Download the files if they dont exist already
        if not filename.is_file():
            print(f"Downloading now: {file_dictionary[df]['url']}")
            _ , _ = urllib.request.urlretrieve(url=file_dictionary[df]["url"], filename=filename)

In [5]:
downloadDataFiles(smass_data_files)

In [6]:
# Untar the spectra data
tar = tarfile.open(os.path.join(core_path, data_path, "smass2data.tar.gz"), "r:gz")
tar.extractall(os.path.join(core_path, data_path))
tar.close()